In [ ]:
import pandas as pd
import itertools

# Read the CSV file
data = pd.read_csv('/Users/abdkahhaleh/Downloads/data.csv')

# Extract unique units for each metric
area_units = data[data['Metric'] == 'Area']['Unit to Convert From'].unique()
energy_units = data[data['Metric'] == 'Energy']['Unit to Convert From'].unique()
mass_units = data[data['Metric'] == 'Mass']['Unit to Convert From'].unique()
volume_units = data[data['Metric'] == 'Volume']['Unit to Convert From'].unique()

# Create a dictionary for conversion rates as strings
conversion_dict = {(row['Unit to Convert From'], row['Unit to Convert To']): str(row['Conversion Rate'])
                   for _, row in data.iterrows()}

# Function to get conversion rate as a string and check if it's not equal to 1
def get_conversion_rate_as_string(from_unit, to_unit):
    rate_str = conversion_dict.get((from_unit, to_unit), conversion_dict.get((to_unit, from_unit), "1"))
    return rate_str if rate_str != "1" else None

# Generate combinations and conversion formulas as strings
combinations = []

# Process for GHGI, EUI, Carbon Emission Factor, Fuel Energy Content, CO2 Emission Intensity
metrics = [
    ('GHGI', mass_units, area_units),
    ('EUI', energy_units, area_units),
    ('Carbon Emission Factor', mass_units, energy_units),
    ('Fuel Energy Content', energy_units, volume_units),
    ('CO2 Emission Intensity', mass_units, volume_units)
]

for metric, units1, units2 in metrics:
    for from_unit1, from_unit2 in itertools.product(units1, units2):
        for to_unit1, to_unit2 in itertools.product(units1, units2):
            rate_from1 = get_conversion_rate_as_string(from_unit1, to_unit1)
            rate_from2 = get_conversion_rate_as_string(from_unit2, to_unit2)
            if rate_from1 and rate_from2:
                formula = f'"={rate_from1}/{rate_from2}"'
                combinations.append([metric, f"{from_unit1}/{from_unit2}", f"{to_unit1}/{to_unit2}", formula])

# Convert to DataFrame
combinations_df = pd.DataFrame(combinations, columns=['Metric', 'Unit to Convert From', 'Unit to Convert To', 'Conversion Formula'])

# Save to CSV
output_file_path = '/Users/abdkahhaleh/Downloads/Generated_Conversions.csv'
combinations_df.to_csv(output_file_path, index=False)
